In [1]:
import pandas as pd
import numpy as np
import math

# Utils

In [2]:
def train_test_split(data, ratio=0.8):
    train_size = int(math.floor(len(data) * ratio))
    train_data = data[:train_size]
    test_data = data[train_size:]
    return train_data, test_data

def remove_outliers_iqr(data):
    df = data.copy()
    Q1 = df[df>0].quantile(0.05)
    Q3 = df[df>0].quantile(0.95)
    IQR = Q3 - Q1
    lower_bound = Q1 - 0.5 * IQR
    upper_bound = Q3 + 1.0 * IQR
    df[(df < lower_bound) | (df > upper_bound)] = np.nan
    return df

def normalize_data(data):
    df = data.copy()

    for col in df.columns:
        min_val = df[col].min()
        max_val = df[col].max()
        range_val = max_val - min_val

        if range_val != 0:
            df[col] = (df[col] - min_val) / range_val
        else:
            df[col] = 0
    return df

def interpolate_data(data):
    data = data.fillna(method='ffill').fillna(method='bfill')
    return data

def filter_data(data, threshold=0.95):
    min_non_na_count = int(data.shape[0] * threshold)
    cleaned_data = data.dropna(thresh=min_non_na_count, axis=1)
    return cleaned_data

In [3]:
def encode_timestamp(data, timestamp_col = "Date"):
    result = data.copy()
    
    if not pd.api.types.is_datetime64_any_dtype(result[timestamp_col]):
        result[timestamp_col] = pd.to_datetime(result[timestamp_col], format="%d/%m/%Y %H:%M:%S")
    
    result['year'] = result[timestamp_col].dt.year
    result['quarter'] = result[timestamp_col].dt.quarter
    result['month'] = result[timestamp_col].dt.month
    result['week_of_year'] = result[timestamp_col].dt.isocalendar().week
    result['day_of_year'] = result[timestamp_col].dt.dayofyear
    result['day_of_month'] = result[timestamp_col].dt.day
    result['day_of_week'] = result[timestamp_col].dt.dayofweek
    result['is_weekend'] = result['day_of_week'].isin([5, 6]).astype(int)
    result['hour'] = result[timestamp_col].dt.hour
    result['minute'] = result[timestamp_col].dt.minute
    
    result['day_of_week_sin'] = np.sin(2 * np.pi * result['day_of_week'] / 7)
    result['day_of_week_cos'] = np.cos(2 * np.pi * result['day_of_week'] / 7)
    
    result['month_sin'] = np.sin(2 * np.pi * result['month'] / 12)
    result['month_cos'] = np.cos(2 * np.pi * result['month'] / 12)
    
    result['hour_sin'] = np.sin(2 * np.pi * result['hour'] / 24)
    result['hour_cos'] = np.cos(2 * np.pi * result['hour'] / 24)
    
    result['minute_sin'] = np.sin(2 * np.pi * result['minute'] / 60)
    result['minute_cos'] = np.cos(2 * np.pi * result['minute'] / 60)
    
    conditions = [
        (result['hour'] >= 5) & (result['hour'] < 12),
        (result['hour'] >= 12) & (result['hour'] < 17),
        (result['hour'] >= 17) & (result['hour'] < 22),
        (result['hour'] >= 22) | (result['hour'] < 5)
    ]
    values = [0, 1, 2, 3]  
    result['time_of_day'] = np.select(conditions, values)
    
    conditions = [
        (result['month'].isin([12, 1, 2])),
        (result['month'].isin([3, 4, 5])),
        (result['month'].isin([6, 7, 8])),
        (result['month'].isin([9, 10, 11]))
    ]
    values = [0, 1, 2, 3]  
    result['season'] = np.select(conditions, values)
    
    result = pd.get_dummies(result, columns=['time_of_day', 'season'], prefix=['tod', 'season'])
    
    if timestamp_col in result.columns:
        result = result.drop(columns=[timestamp_col])
        
    return result
    

# Meter Data

In [4]:
df = pd.read_csv("./data/customer_led_network_revolution/TrialMonitoringDataHH.csv", usecols=["Date and Time of capture", "Location ID", "Parameter"], index_col="Date and Time of capture", engine="c")
df_T = df.pivot_table(index=df.index, columns='Location ID', values='Parameter')
print(df_T.shape)
df_T.head()

(42480, 8798)


Location ID,1,3,4,5,6,8,9,10,11,12,...,9191,9192,9193,9194,9195,9196,9197,9198,9199,9200
Date and Time of capture,,,,,,,,,,,,,,,,,,,,,
01/01/2012 00:00:00,0.082,0.202,0.219,0.004,0.181,NaN,0.307,0.013,0.216,0.182,...,0.048,0.086,0.156,0.481,0.056,0.078,0.144,NaN,0.042,0.110
01/01/2012 00:30:00,0.082,0.096,0.098,0.035,0.210,NaN,0.155,0.003,0.220,0.182,...,0.069,0.104,0.216,0.484,0.095,0.118,0.146,NaN,0.045,0.059
01/01/2012 01:00:00,0.084,0.108,0.108,0.003,0.075,NaN,0.071,0.017,0.181,0.020,...,0.034,0.107,0.147,0.498,0.047,0.086,0.149,NaN,0.090,0.062
01/01/2012 01:30:00,0.083,0.069,0.105,NaN,0.053,NaN,0.091,0.001,0.189,0.061,...,0.027,0.090,0.134,0.495,0.041,0.090,0.124,NaN,0.085,0.052
01/01/2012 02:00:00,0.277,0.115,0.086,0.036,0.072,NaN,0.093,0.016,0.210,0.008,...,0.036,0.098,0.163,0.491,0.056,0.112,0.163,NaN,0.039,0.062


In [5]:
print(df_T.shape)
filtered_df = filter_data(df_T)
filtered_df.shape

(42480, 8798)


(42480, 4468)

In [6]:
no_outlier_df = remove_outliers_iqr(filtered_df)

In [7]:
train, test = train_test_split(no_outlier_df)
val, test = train_test_split(test)

In [8]:
train.shape, val.shape, test.shape

((33984, 4468), (6796, 4468), (1700, 4468))

In [9]:
imputed_train = interpolate_data(train)
imputed_val = interpolate_data(val)
imputed_test = interpolate_data(test)

print(imputed_train.isnull().sum().sum(), imputed_test.isnull().sum().sum())

C:\Users\Arne\AppData\Local\Temp\ipykernel_22240\3228508037.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill').fillna(method='bfill')


0 0


In [10]:
norm_train = normalize_data(imputed_train)
norm_val = normalize_data(imputed_val)
norm_test = normalize_data(imputed_test)

print(norm_train.min().min(), norm_train.max().max())
print(norm_test.min().min(), norm_test.max().max())

0.0 1.0
0.0 1.0


In [13]:
norm_train.to_csv("./data/customer_led_network_revolution/preprocessed/train.csv")
norm_val.to_csv("./data/customer_led_network_revolution/preprocessed/val.csv")
norm_test.to_csv("./data/customer_led_network_revolution/preprocessed/test.csv")

# Time data

In [14]:
train_idx = norm_train.index.values
val_idx = norm_val.index.values
test_idx = norm_test.index.values

In [15]:
train_idx_df = pd.DataFrame(train_idx, columns=["Date"])
val_idx_df = pd.DataFrame(val_idx, columns=["Date"])
test_idx_df = pd.DataFrame(test_idx, columns=["Date"])

train_idx_df.head()

,Date
0,01/01/2012 00:00:00
1,01/01/2012 00:30:00
2,01/01/2012 01:00:00
3,01/01/2012 01:30:00
4,01/01/2012 02:00:00


In [16]:
train_time_cond_df = encode_timestamp(train_idx_df)
val_time_cond_df = encode_timestamp(val_idx_df)
test_time_cond_df = encode_timestamp(test_idx_df)

train_time_cond_df.head()

,year,quarter,month,week_of_year,day_of_year,day_of_month,day_of_week,is_weekend,hour,minute,...,minute_sin,minute_cos,tod_0,tod_1,tod_2,tod_3,season_0,season_1,season_2,season_3
0,2012,1,1,52,1,1,6,1,0,0,...,0.000000e+00,1.0,False,False,False,True,True,False,False,False
1,2012,1,1,52,1,1,6,1,0,30,...,5.665539e-16,-1.0,False,False,False,True,True,False,False,False
2,2012,1,1,52,1,1,6,1,1,0,...,0.000000e+00,1.0,False,False,False,True,True,False,False,False
3,2012,1,1,52,1,1,6,1,1,30,...,5.665539e-16,-1.0,False,False,False,True,True,False,False,False
4,2012,1,1,52,1,1,6,1,2,0,...,0.000000e+00,1.0,False,False,False,True,True,False,False,False


In [17]:
for col in train_time_cond_df.columns:
    if train_time_cond_df[col].dtype == "bool":
        train_time_cond_df[col] = train_time_cond_df[col].astype("int32")
        
for col in val_time_cond_df.columns:
    if val_time_cond_df[col].dtype == "bool":
        val_time_cond_df[col] = val_time_cond_df[col].astype("int32")

for col in test_time_cond_df.columns:
    if test_time_cond_df[col].dtype == "bool":
        test_time_cond_df[col] = test_time_cond_df[col].astype("int32")

train_time_cond_df.head()

,year,quarter,month,week_of_year,day_of_year,day_of_month,day_of_week,is_weekend,hour,minute,...,minute_sin,minute_cos,tod_0,tod_1,tod_2,tod_3,season_0,season_1,season_2,season_3
0,2012,1,1,52,1,1,6,1,0,0,...,0.000000e+00,1.0,0,0,0,1,1,0,0,0
1,2012,1,1,52,1,1,6,1,0,30,...,5.665539e-16,-1.0,0,0,0,1,1,0,0,0
2,2012,1,1,52,1,1,6,1,1,0,...,0.000000e+00,1.0,0,0,0,1,1,0,0,0
3,2012,1,1,52,1,1,6,1,1,30,...,5.665539e-16,-1.0,0,0,0,1,1,0,0,0
4,2012,1,1,52,1,1,6,1,2,0,...,0.000000e+00,1.0,0,0,0,1,1,0,0,0


In [18]:
skip_cols = []
skip_cols.extend([col for col in train_time_cond_df.columns if col.startswith('tod_') or col.startswith('season_')])
train_time_norm_df = train_time_cond_df.drop(skip_cols, axis=1)
train_time_norm_df = normalize_data(train_time_norm_df)
train_time_norm_df = pd.concat([train_time_norm_df, train_time_cond_df[skip_cols]], axis=1)
train_time_norm_df.head()

,year,quarter,month,week_of_year,day_of_year,day_of_month,day_of_week,is_weekend,hour,minute,...,minute_sin,minute_cos,tod_0,tod_1,tod_2,tod_3,season_0,season_1,season_2,season_3
0,0.5,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.000000,0.0,...,0.0,1.0,0,0,0,1,1,0,0,0
1,0.5,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.000000,1.0,...,1.0,0.0,0,0,0,1,1,0,0,0
2,0.5,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.043478,0.0,...,0.0,1.0,0,0,0,1,1,0,0,0
3,0.5,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.043478,1.0,...,1.0,0.0,0,0,0,1,1,0,0,0
4,0.5,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.086957,0.0,...,0.0,1.0,0,0,0,1,1,0,0,0


In [19]:
skip_cols = []
skip_cols.extend([col for col in val_time_cond_df.columns if col.startswith('tod_') or col.startswith('season_')])
val_time_norm_df = val_time_cond_df.drop(skip_cols, axis=1)
val_time_norm_df = normalize_data(val_time_norm_df)
val_time_norm_df = pd.concat([val_time_norm_df, val_time_cond_df[skip_cols]], axis=1)
val_time_norm_df.head()

,year,quarter,month,week_of_year,day_of_year,day_of_month,day_of_week,is_weekend,hour,minute,...,minute_sin,minute_cos,tod_0,tod_1,tod_2,tod_3,season_0,season_1,season_2,season_3
0,0.0,0.333333,0.454545,0.4375,0.443787,0.0,0.833333,1.0,0.000000,0.0,...,0.0,1.0,0,0,0,1,0,0,1,0
1,0.0,0.333333,0.454545,0.4375,0.443787,0.0,0.833333,1.0,0.000000,1.0,...,1.0,0.0,0,0,0,1,0,0,1,0
2,0.0,0.333333,0.454545,0.4375,0.443787,0.0,0.833333,1.0,0.043478,0.0,...,0.0,1.0,0,0,0,1,0,0,1,0
3,0.0,0.333333,0.454545,0.4375,0.443787,0.0,0.833333,1.0,0.043478,1.0,...,1.0,0.0,0,0,0,1,0,0,1,0
4,0.0,0.333333,0.454545,0.4375,0.443787,0.0,0.833333,1.0,0.086957,0.0,...,0.0,1.0,0,0,0,1,0,0,1,0


In [20]:
skip_cols = []
skip_cols.extend([col for col in test_time_cond_df.columns if col.startswith('tod_') or col.startswith('season_')])
test_time_norm_df = test_time_cond_df.drop(skip_cols, axis=1)
test_time_norm_df = normalize_data(test_time_norm_df)
test_time_norm_df = pd.concat([test_time_norm_df, test_time_cond_df[skip_cols]], axis=1)
test_time_norm_df.head()

,year,quarter,month,week_of_year,day_of_year,day_of_month,day_of_week,is_weekend,hour,minute,...,minute_sin,minute_cos,tod_0,tod_1,tod_2,tod_3,season_0,season_1,season_2,season_3
0,1.0,0.333333,0.363636,0.411765,0.355224,0.0,0.5,0.0,0.608696,0.0,...,0.0,1.0,0,1,0,0,0,1,0,0
1,1.0,0.333333,0.363636,0.411765,0.355224,0.0,0.5,0.0,0.608696,1.0,...,1.0,0.0,0,1,0,0,0,1,0,0
2,1.0,0.333333,0.363636,0.411765,0.355224,0.0,0.5,0.0,0.652174,0.0,...,0.0,1.0,0,1,0,0,0,1,0,0
3,1.0,0.333333,0.363636,0.411765,0.355224,0.0,0.5,0.0,0.652174,1.0,...,1.0,0.0,0,1,0,0,0,1,0,0
4,1.0,0.333333,0.363636,0.411765,0.355224,0.0,0.5,0.0,0.695652,0.0,...,0.0,1.0,0,1,0,0,0,1,0,0


In [21]:
train_time_norm_df.to_csv("./data/customer_led_network_revolution/preprocessed/cond_train.csv")
val_time_norm_df.to_csv("./data/customer_led_network_revolution/preprocessed/cond_val.csv")
test_time_norm_df.to_csv("./data/customer_led_network_revolution/preprocessed/cond_test.csv")